# 네이버 카페 contents 크롤링 
제목/ 닉네임/ 날짜 / 내용/ 댓글 데이터 가져오기

In [1]:
import requests
import pandas as pd

import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import urllib.request
from bs4 import BeautifulSoup

### 2015 네이버 키워드 '쌀'
1차 바이럴제거를 위해 검색 설정

: 쌀 -업체, -제공받, -지원받, -원고료, -무상, -협찬, -소정의, -사료, -고래밥, -길냥이, -고양이, -강아지
https://search.naver.com/search.naver?sm=tab_hty.top&where=article&query=%EC%8C%80&oquery=Tkf&tqi=hdzB%2Fdp0YihssA3p2MGssssss3s-479049&nso=so%3Ar%2Cp%3Afrom20150101to20151231

## 01월
https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.01.01&date_to=2015.01.31&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20150101to20150131&nso_open=1&t=0&mson=0&prdtype=0

In [12]:
#쌀
driver = webdriver.Chrome('./chromedriver') 
driver.implicitly_wait(4) # 바로 에러넘기지 말고 기다리는 시간 4초 설정
driver.get('https://search.naver.com/search.naver?where=article&query=%EC%8C%80%20-%EC%97%85%EC%B2%B4%2C%20-%EC%A0%9C%EA%B3%B5%EB%B0%9B%2C%20-%EC%A7%80%EC%9B%90%EB%B0%9B%2C%20-%EC%9B%90%EA%B3%A0%EB%A3%8C%2C%20-%EB%AC%B4%EC%83%81%2C%20-%ED%98%91%EC%B0%AC%2C%20-%EC%86%8C%EC%A0%95%EC%9D%98%2C%20-%EC%82%AC%EB%A3%8C%2C%20-%EA%B3%A0%EB%9E%98%EB%B0%A5%2C%20-%EA%B8%B8%EB%83%A5%EC%9D%B4%2C%20-%EA%B3%A0%EC%96%91%EC%9D%B4%2C%20-%EA%B0%95%EC%95%84%EC%A7%80&ie=utf8&st=rel&date_option=99&date_from=2015.01.01&date_to=2015.01.31&board=&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20150101to20150131&nso_open=1&t=0&mson=0&prdtype=0')
time.sleep(0.5)

In [13]:
# second 동안 스크롤다운 함수
import datetime

def doScrollDown(whileSeconds):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=whileSeconds)
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        if datetime.datetime.now() > end:
            break
        

In [14]:
doScrollDown(100)

In [5]:
# 크롤링할 주소 동적가져오기
article_list = driver.find_elements_by_css_selector('.api_txt_lines.total_tit')
len(article_list)

1000

## 1. 데이터 수집 함수 정의, 한번에 가져오기

In [8]:
def get_Data(driver):
    driver.switch_to.frame('cafe_main') # 프레임 변경
    soup = BeautifulSoup(driver.page_source, 'html.parser') # 페이지 준비

    #제목/ 닉네임/ 날짜 
    title = soup.select('h3.title_text')[0].text.strip() #h3태그에 title_text클래스
    nickname = soup.select('a.nickname')[0].text.strip() #a테그에 nickname클래스
    date = soup.select('span.date')[0].text[:10] #시간제외 날짜까지만 인덱싱하여 가져옴
    
    #내용
    try:
        content = soup.select('div.ContentRenderer')[0].text #div태그에 ContentRenderer클래스
    except:
        pass

    # 댓글 : 여러개 존재하므로 모두가져온다음 text만 뽑음 
    comments_all = soup.select('span.text_comment')
    comment_list =[]
    for comment in comments_all:
        comment = comment.text.strip()
        comment_list.append(comment)

    # 댓글 : 리스트-> 문자열로 저장
    seperator = '||'
    comments = seperator.join(comment_list)

    # 제목/ 닉네임/ 날짜 / 내용/ 댓글 데이터-> 리스트로 저장
    post = [title , nickname, date, content,comments]

    return post


In [9]:
total_data=[] # 총데이터 저장할 리스트

# 카페 개별 페이지 접속
for article in article_list:
    article.click()  # 한페이지에 접속완료

    # 드라이버 윈도우 설정
    change_tab = driver.window_handles[-1]
    driver.switch_to.window(change_tab) 

    try:
        data = get_Data(driver) # 수집
        total_data.append(data)
    except:
        pass

    driver.close() # 한페이지 수집완료

    # 다른 게시글 들어갈 준비 (必)
    change_tab = driver.window_handles[-1]
    driver.switch_to.window(change_tab)


## 데이터 엑셀파일에 저장

In [10]:
# title , nickname, date, content,comments 데이터프레임으로 저장
df = pd.DataFrame(total_data,columns=['title','nickname','date','content','comments'])
df

,title,nickname,date,content,comments
0,공구쌀 후기,메이주,2015.01.27,세종시로 올해 1월 10일 이사했습니다.어제 쌀이 똑 떨어졌는데 마침 까페에 쌀공구...,ㅎㅎ밥에 꿀발라~~~\n꿀바른것같은데 꿀맛이 안나고 밥맛이나죠\n꼬소름한 밥맛 ㅎㅎ...
1,빨간쌀 필요하신분이요^^,악기하는 여자,2015.01.04,저희 큰아버지께서 쌀도매를 하세요^^빨간쌀 이예요.저도 작녁부터 계속 먹고있는데요^...,네^^일반쌀 검정쌀 처럼 섞어드시는거구요♡\n구입 하시는분 많으심~~메타로 가져다 ...
2,쌀맛나는세상님 현미쌀 구입기^^,축복맘,2015.01.13,제가 변비가 심해서 현미쌀100%를 먹어야 되서 쌀맛나는세상님 현미쌀을 구매 했어요...,현미밥 어떻게하면 맛있나요?||밥솥이 중요한것 같아요~~ 밥솥 고장 나서 새로...
3,쌀찐빵 더 맛있게 먹기^^,은혜맘,2015.01.12,그동안 계속 공구후기며 공구게시판에서 보던 쌀찐빵을 구입했어요 아기간식도 요즘 고민...,||후기이벤트 당첨소식이 늦었습니다~!\n\n^^ 귀한후기 감사드립니다\n훈제기에 ...
4,당근쌀머핀 구워보아요~,호호바다,2015.01.11,심심한 일요일 오후.당근쌀머핀 만들어 보았어요~당근이랑 우유는 휘리릭 갈아주고밀가루...,강력말고 일반 쌀가루 써도 돼요? \n맛있어보여요 ~~^*^||원래 머핀은 박력쌀가...
...,...,...,...,...,...
612,김밥 파뤼~~^^,붕이양75,2015.01.27,낭군님의 요청으로 오늘 저녁은 김밥 ㅋ근데 많다 10줄...김밥 재료를 사면 ...,맛있겠어용.ㅋ나머지다넣은한줄은더맛날듯ㅋ||ㅋ 그건 제가 호로록했네요 ㅋㅋ||10줄싸...
613,쌀떡볶이,꽃돌핀순수우유,2015.01.31,먹던중한것,맛난거..||꺄아아아아\n오늘 떡볶이 해먹어야겠당ㅋㅋ 군침이...||맛있어보여요 ㅎ...
614,민성이가 주말동안 별 차도가 없었어요 ㅠㅠ,민성이맘,2015.01.26,"주말 잘 보내셨나요~?^^민성이는 금욜부터 주말동안 먹은거는 쌀미음, 바나나, 찐단...",민성이가 힘든 주말을 보냈나봐요..\n민성이가 지금 서있는 곳이 놀이방매트인가요?\...
615,쌀과자-완료-,피아노치는 서산댁,2015.01.28,8개월이후한봉당 2300원짜구요모두 2000원이요유통기한2015 05 16읍내현대~,저용


In [11]:
# 엑셀 파일로 저장
writer = pd.ExcelWriter('naver_cafe_ssal_contents_2015.xlsx')
df.to_excel(writer,index =False)
writer.save()